#Fouloscopie

##Boid Simulation

In [ ]:
import pygame
import random
import math

#initialises the pygame
pygame.init()

clock = pygame.time.Clock()
#parameters
windowWidth, windowHeight = 1000, 800
margin_x, margin_y = 600, 480

boidNum = 150
boidList = []

white = (255, 255, 255)
black = (0, 0, 0)

maxspeed = 4
minspeed = 1.5

turnfactor = 0.05
matchingfactor = 0.5
centeringfactor = 0.00001
avoidfactor = 0.5

#ranges
visualRange = 50
protectedRange = 18

leftmargin = (windowWidth - margin_x) / 2
rightmargin = windowWidth - (windowWidth - margin_x) / 2
bottommargin = (windowHeight - margin_y) / 2
topmargin = windowHeight - (windowHeight - margin_y) / 2

#creation of screen, with the inside being the width and height
screen = pygame.display.set_mode((windowWidth, windowHeight))

#Boid Class (why are classes needed?)
class Boid: 
    def __init__(self, x, y):
        self.x = x  #starting x pos
        self.y = y  # starting y pos

        #velocity
        self.vx = random.uniform(minspeed, maxspeed) * random.choice([-1,1])
        self.vy = random.uniform(minspeed, maxspeed) * random.choice([-1,1])

        #acceleration
        self.ax = self.vx * random.uniform(0, 6)
        self.ay = self.vy * random.uniform(0, 6)

        

    def update_position(self):
        self.x += self.vx
        self.y += self.vy


        if self.x < leftmargin:
            self.vx += turnfactor 
        if self.x > rightmargin:
            self.vx -= turnfactor 
        if self.y < bottommargin:
            self.vy += turnfactor 
        if self.y > topmargin:
            self.vy -= turnfactor

        speed = math.hypot(self.vx, self.vy)

        if speed>maxspeed:
            self.vx = (self.vx/speed)*maxspeed
            self.vy = (self.vy/speed)*maxspeed
        elif speed<minspeed:
            self.vx = (self.vx/speed)*minspeed
            self.vy = (self.vy/speed)*minspeed

    def separation(self):
        close_dx = 0
        close_dy = 0

        for otherboid in boidList:
            if otherboid == self:
                continue
            else:
                dx = self.x - otherboid.x
                dy = self.y - otherboid.y
                distance = math.hypot(dx,dy)
                if 0 <= distance < protectedRange:
                    close_dx += dx/distance
                    close_dy += dy/distance
        self.vx += close_dx*avoidfactor
        self.vy += close_dy*avoidfactor

    def alignment(self):
        xvel_avg = 0
        yvel_avg = 0
        neighbouring_boids = 0

        for otherboid in boidList:
            if otherboid == self:
                continue
        else:
            dx = self.x - otherboid.x
            dy = self.y - otherboid.y
            distance = math.hypot(dx,dy)

            if distance < visualRange:
                xvel_avg += otherboid.vx
                yvel_avg += otherboid.vy
                neighbouring_boids += 1
                
            if neighbouring_boids>0:
                xvel_avg = xvel_avg/neighbouring_boids
                yvel_avg = yvel_avg/neighbouring_boids
        
        dvx = xvel_avg - self.vx
        dvy = yvel_avg - self.vy
        boid.vx += dvx*matchingfactor
        boid.vy += dvy*matchingfactor
    
    def cohesion(self):
        xpos_avg = 0
        ypos_avg = 0
        neighbouring_boids = 0

        for otherboid in boidList:
            if otherboid == self:
                continue
            dx = self.x - otherboid.x
            dy = self.y - otherboid.y
            distance = math.hypot(dx,dy)

            if distance < visualRange:
                xpos_avg += otherboid.x
                ypos_avg += otherboid.y
                neighbouring_boids += 1

            if neighbouring_boids>0:
                xpos_avg = xpos_avg/neighbouring_boids
                ypos_avg = ypos_avg/neighbouring_boids
        
        self.vx += (xpos_avg - self.x)*centeringfactor
        self.vy += (ypos_avg - self.y)*centeringfactor

    # Check if the boid is outside the margin
        #need to continually draw the shape for each frame
    def draw(self):
        #circle: pygame.draw.circle(surface, colour, center, radius, width=0)
        pygame.draw.circle(screen, white, (int(self.x), int(self.y)), 3.5)

#create a list of boids, iterated outside of class so it can be referenced by the rest of code
for i in range(boidNum):
    boidList.append(Boid(random.uniform(leftmargin, rightmargin), random.uniform(bottommargin, topmargin)))

#Game Loop
running = True
while running:
    clock.tick(60)
    for event in pygame.event.get():
        if event.type == pygame.QUIT: #pygame.QUIT is an event
            running = False
          

    #update the position of a list of boids
    for boid in boidList:
        boid.update_position()
        boid.separation()
        boid.alignment()
        boid.cohesion()

    #makes sure that boids don't leave continuous trail
    screen.fill(black)

    #draws the boid at the new position
    for boid in boidList:
        boid.draw()
    pygame.display.update()
